> # 필요한 패키지 불러오기

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import Imputer
from sklearn.preprocessing import PolynomialFeatures 
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import VarianceThreshold  
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import xgboost as xgb 
import warnings
warnings.filterwarnings('ignore')

import os
print(os.listdir("../input"))

# 디버깅할때 데이터 줄여서 읽을 수 있도록 

In [ ]:
# debug = True

# if debug:
#     nrows = 50000
# else:
#     nrows = None

In [ ]:
# %%time
# kaggle = '../input/'
# train = pd.read_csv(kaggle +'train.csv', nrows = nrows,parse_dates=['Dates'])
# test = pd.read_csv(kaggle + 'test.csv', nrows = nrows, parse_dates=['Dates'], index_col='Id')
# test = test.sample(frac = 0.1)

# 필요한 패키지를 불러오고 데이터를 불러온다.

In [ ]:
# %%time
# kaggle = '../input/'
# train = pd.read_csv(kaggle +'train.csv', nrows = nrows,parse_dates=['Dates'])
# test = pd.read_csv(kaggle + 'test.csv', nrows = nrows, parse_dates=['Dates'], index_col='Id')
# test = test.sample(frac = 0.1)import pandas as pd

import numpy as np
from sklearn.preprocessing import LabelEncoder
import lightgbm as lgb
pd.options.display.max_columns=100
train = pd.read_csv('../input/train.csv', parse_dates=['Dates'])
test = pd.read_csv('../input/test.csv', parse_dates=['Dates'], index_col='Id')

def feature_engineering(data):
    data['Date'] = pd.to_datetime(data['Dates'].dt.date)
    data['n_days'] = (data['Date'] - data['Date'].min()).apply(lambda x: x.days)
    data['Day'] = data['Dates'].dt.day
    data['DayOfWeek'] = data['Dates'].dt.weekday
    data['Month'] = data['Dates'].dt.month
    data['Year'] = data['Dates'].dt.year
    data['Hour'] = data['Dates'].dt.hour
    data['Minute'] = data['Dates'].dt.minute
    data['Block'] = data['Address'].str.contains('block', case=False).apply(lambda x: 1 if x == True else 0)
    data["X_Y"] = data["X"] - data["Y"]
    data["XY"] = data["X"] + data["Y"]
    data.drop(columns=['Dates','Date','Address'], inplace=True)
    return data
train = feature_engineering(train)
test = feature_engineering(test)
train.drop(columns=['Descript','Resolution'], inplace=True)

# 'Category'변수에서 지울 것 지우기
- eda시에 category 변수에서 드물게 나오는 것들을 확인 할 수 있었음 
- 그러므로 우리의 classifier가 좀 더 잘 작동할 수 있도록 지워주겠음

In [ ]:

# rare_cats = set(['FAMILY OFFENSES', 'BAD CHECKS', 'BRIBERY', 'EXTORTION',
#        'SEX OFFENSES NON FORCIBLE', 'GAMBLING', 'PORNOGRAPHY/OBSCENE MAT',
#        'TREA'])
# all_cats = set(train['Category'].unique())
# common_cats = all_cats-rare_cats

In [ ]:

# train= train[train['Category'].isin(common_cats)]
# train = train.reset_index(drop = True)

# PCA로 위도 경도 처리
- pca는 차원 축소에 주로 사용되는 알고리즘
- X,Y(위경도)는 2차원 
- 그러므로 축소하지 않고 차원을 유지하면서 pca.transform 을 실시 
- 그러면 원본 데이터를 변형해서 새로운 피쳐를 만들 수 있음 

In [ ]:
for i in train,test:
    data = i
  
    coord = data[['X','Y']]
    pca = PCA(n_components=2)
    pca.fit(coord)

    coord_pca = pca.transform(coord)

    data['coord_pca1'] = coord_pca[:, 0]
    data['coord_pca2'] = coord_pca[:, 1]

In [ ]:
train.head()

# K-means 군집 -위도,경도
- k-means 군집은 군집에 사용되는 unsupervised learning 알고리즘
- k-means 군집을 하면 가까운 지역끼리 군집이되기 때문에 zipcode 같이 유사한 개념의 새로운 피쳐를 만들 수 있다

### k를 어떻게 결정할까?
- k-means 군집에서는 k를 어떻게 정하는지가 중요하다.
- 일반적으로는 k값을 늘리면서, 군집간의 거리의 합인 inertia가 급격하게 떨어지는 구간을 k값으로 한다.
- 이런 방법을 elbow method라고 한다 

- 하지만 여기서는 그렇게 결과 값이 좋게 나타나지 않았으므로 사용하지 않는다

In [ ]:
# inertia_arr = []

# k_range = range(2, 16)

# for k in k_range:
#     kmeans = KMeans(n_clusters=k, random_state=42).fit(coord)
 
#     # Sum of distances of samples to their closest cluster center
#     interia = kmeans.inertia_
#     print ("k:",k, " cost:", interia)
#     inertia_arr.append(interia)
    
# inertia_arr = np.array(inertia_arr)

# plt.plot(k_range, inertia_arr)
# plt.vlines(5, ymin=inertia_arr.min()*0.9999, ymax=inertia_arr.max()*1.0003, linestyles='--', colors='b')
# plt.title('Elbow Method')
# plt.xlabel('Number of clusters')
# plt.ylabel('Inertia');

In [ ]:

# for i in train,test:
#     data = i
#     coord = data[['X','Y']]
#     # kmeans for lat, long
#     kmeans = KMeans(n_clusters=5, random_state=42).fit(coord)
#     coord_cluster = kmeans.predict(coord)
#     data['coord_cluster'] = coord_cluster
#     data['coord_cluster'] = data['coord_cluster'].map(lambda x: 'c_' + str(x).rjust(2, '0'))


# one-hot encoding 
- one-hot encoding으로 categorical 전처리
- 모두 [0,1,0,0] , [0,0,0,1] 이러한 형태로 바꿔준다 
- 하지만 이번 분석에서는 ___원핫인코딩을 했을때 보다 단순히 카테고리 변수로 모델을 훈련시켰을때 성능이 더 좋았기에 원 핫으로 처리하지 않음___

In [ ]:
# v = cat_val
# for f in v:
#     dist_values = train[f].value_counts().shape[0]
#     print('Variable {} has {} distinct values'.format(f, dist_values))

In [ ]:
# print('더미 전 트레인 변수의 수'.format(train.shape[1]))
# train = pd.get_dummies(train,columns=v, drop_first=True)
# print('터미 후 트레인 변수의 수 '.format(train.shape[1]))

# LabelEncoder()을 이용해 categorical 변수 처리

In [ ]:
le1 = LabelEncoder()
train['PdDistrict'] = le1.fit_transform(train['PdDistrict'])
test['PdDistrict'] = le1.transform(test['PdDistrict'])

le2 = LabelEncoder()
X = train.drop(columns=['Category'])
y= le2.fit_transform(train['Category'])

In [ ]:
train.info()

# sklearn의 PolynomialFeatures 을 이용해 변수끼리 interaction
- sklearn의 PolynomialFeatures 함수를 이용해 피쳐끼리 곱해줘 새로운 피쳐를 만들어 준다.
- 서로 상관이 없어 보이던 피쳐들도 서로 곱해지면 유의미한 변수가 될 수 있음
- ___하지만 여기에서는 이 작업을 해도 스코어가 비슷했음___
- ___그러므로 시간을 절약하고, 컴퓨팅 파워를 절약하기위해 생략___

### 위경도 데이터끼리 interaction

In [ ]:

# #모둔 조합에 대해서 degree 2로 제곱(중첩되는 것은 날려줌, 그러넫 자신을 제곱하는 항이 있냐 없냐는 F하면 제곱
# #하는데 의미 없다고 생각함),
# #인털엑션하고 나머지 바이어스는 뺀다. #self interaction을 

# v1 = ['X','Y','coord_pca1','coord_pca2']

# poly = PolynomialFeatures(degree=2, interaction_only=False, include_bias=False)

# interactions = pd.DataFrame(data=poly.fit_transform(train[v1]), columns=poly.get_feature_names(v1))
# #poly(곱하는 거에)train[v]를 fit시키고 data로 지정해주고,
# #컬럼을 poly식에서 피쳐 네임을 뽑아준다.

# #
# interactions.drop(v1, axis=1, inplace=True)  # Remove the original columns


# # Concat the interaction variables to the train data
# print('Before creating interactions we have {} variables in train'.format(train.shape[1]))

# #여기서 중요한거는 엑스트라 변수를 어디다가 더해주냐인데 axis가 1(열)로 해줘야지 옆으로 붙음 axis=0 이면 아래로 붙어서
# train = pd.concat([train, interactions], axis=1) 
# print('After creating interactions we have {} variables in train'.format(train.shape[1]))

 ### 시간 관련 데이터끼리 interaction****

In [ ]:

# #모둔 조합에 대해서 degree 2로 제곱(중첩되는 것은 날려줌, 그러넫 자신을 제곱하는 항이 있냐 없냐는 F하면 제곱
# #하는데 의미 없다고 생각함),
# #인털엑션하고 나머지 바이어스는 뺀다. #self interaction을 

# # v = ['X','Y','coord_pca1','coord_pca2']
# v = ['n_days','Day','Month','Year','Hour','Minute']
# poly = PolynomialFeatures(degree=2, interaction_only=False, include_bias=False)

# interactions = pd.DataFrame(data=poly.fit_transform(train[v]), columns=poly.get_feature_names(v))
# #poly(곱하는 거에)train[v]를 fit시키고 data로 지정해주고,
# #컬럼을 poly식에서 피쳐 네임을 뽑아준다.

# #
# interactions.drop(v, axis=1, inplace=True)  # Remove the original columns


# # Concat the interaction variables to the train data
# print('Before creating interactions we have {} variables in train'.format(train.shape[1]))

# #여기서 중요한거는 엑스트라 변수를 어디다가 더해주냐인데 axis가 1(열)로 해줘야지 옆으로 붙음 axis=0 이면 아래로 붙어서
# #바보같아질 수 있음 
# train = pd.concat([train, interactions], axis=1) 
# print('After creating interactions we have {} variables in train'.format(train.shape[1]))

In [ ]:
train.head()

# lightGBM

In [ ]:
train_data = lgb.Dataset(X, label=y, categorical_feature=['PdDistrict', ])
params = {'boosting':'gbdt',
          'objective':'multiclass',
          'num_class':39,
          'max_delta_step':0.9,
          'min_data_in_leaf': 20,
          'learning_rate': 0.29,
          'max_bin': 501,
          'num_leaves': 41,
          'verbose' : 1}

bst = lgb.train(params, train_data, 120)
predictions_lgb = bst.predict(test)



In [ ]:
train_data = lgb.Dataset(X, label=y, categorical_feature=['PdDistrict', ])
params = {'boosting':'gbdt',
          'objective':'multiclass',
          'num_class':39,
          'max_delta_step':0.9,
          'min_data_in_leaf': 4,
          'learning_rate': 0.29,
          'max_bin': 501,
          'num_leaves': 41,
          'verbose' : 1}

bst = lgb.train(params, train_data, 120)
predictions_lgb1 = bst.predict(test)



In [ ]:
predictions_lgb1

In [ ]:
predictions_lgb

In [ ]:
con2 = (predictions_lgb + predictions_lgb1)/2
con2

# XGboost

In [ ]:

# # train_xgb = xgb.DMatrix(X, label=y)
# # test_xgb  = xgb.DMatrix(test)
# dtrain = xgb.DMatrix(X, label=y)
# dtest  = xgb.DMatrix(test) 

In [ ]:
# params = {
#     'max_depth': 5, 
#     'eta': 0.3,  
#     'num_boost_rounds' : 150 ,
#     'silent': 1, 
#     'objective': 'multi:softprob',  
#     'eval_metric' : 'mlogloss',
#     'learning_data' : 0.07,
#     'num_class': 39,
#     'min_child_weight':1,
# }

In [ ]:
# model_xgb = xgb.train(params, dtrain, 10)
# predictions_xgb = model_xgb.predict(dtest)



In [ ]:
#하이퍼 파라미터 테스트의 수행 속도를 향상시키기 위해 n_estimators 를 100으로 감소


# GridSearch
- 하이퍼 파라미터를 튜닝하는데 최적의 파라미터를 찾아준다
- 단점으로 시간이 매우 오래 걸린다.

In [ ]:
# predictions_xgb

In [ ]:
# con1 = (predictions_lgb + predictions_xgb)/2
# con1

In [ ]:
submission = pd.DataFrame(con2, columns=le2.inverse_transform(np.linspace(0, 38, 39, dtype='int16')),
                          index=test.index)
 
submission.to_csv('submission.csv', index='Id')

In [ ]:
submission